In [1]:
# Load the SeqArray package
library(SeqArray)
library(tidyverse)

outdir="/rds/general/user/ah3918/projects/puklandmarkproject/live/Users/Alex/pipelines/eqtl_pipeline_dev"
gds_file="/rds/general/user/ah3918/projects/roche/live/ALEX//PROCESSED_DATA/PROCESSED_GENOTYPE/FINAL/final_geno_440samples.gds"

Loading required package: gdsfmt

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ stringr::fixed() masks SeqArray::fixed()
✖ dplyr::lag()     masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
gds_file="/rds/general/user/ah3918/projects/roche/live/ALEX//PROCESSED_DATA/PROCESSED_GENOTYPE/FINAL/final_geno_440samples.gds"
parallel=FALSE
preprocess=FALSE
autosomalonly=TRUE
minmaf=0.05

In [3]:
genofile=SeqArray::seqOpen(gds_file)

In [5]:
library(GenomicRanges)
# Define the original range
original_range <- GRanges(seqnames = "chr8",
                          ranges = IRanges(start = 100572874, end = 100663585))
                          # Extend the range by 1Mb (1,000,000 base pairs) from each end
extended_start <- start(original_range) - 1000000
extended_end <- end(original_range) + 1000000

# Create the extended GRanges object
extended_range <- GRanges(seqnames = "chr8",
                          ranges = IRanges(start = extended_start, end = extended_end))

# Print the extended range
extended_range

GRanges object with 1 range and 0 metadata columns:
      seqnames             ranges strand
         <Rle>          <IRanges>  <Rle>
  [1]     chr8 99572874-101663585      *
  -------
  seqinfo: 1 sequence from an unspecified genome; no seqlengths

In [6]:
SeqArray::seqSetFilterCond(genofile,maf=minmaf)


[==================================================] 100%, completed, 10s
# of selected variants: 5,224,621


In [7]:
SeqArray::seqSetFilter(genofile,variant.sel=extended_range)

# of selected variants: 3,354


In [9]:
seqExport(genofile, "/rds/general/user/ah3918/projects/puklandmarkproject/live/Users/Alex/pipelines/TEST_DATA/test_geno.gds")

Export to ‘/rds/general/user/ah3918/projects/puklandmarkproject/live/Users/Alex/pipelines/TEST_DATA/test_geno.gds’:
    sample.id (440)  [md5: 19a02026c60ef51016d58081c3b86906]
    variant.id (3,354)  [md5: b40e44716784be1768a9a07167182279]
    position  [md5: 66239dd74f8c12f9d52d98487a9a6909]
    chromosome  [md5: 16a41660c237f7dbcc1498bfad3d63bf]
    allele  [md5: b47dbc34e864012cdda251b95aec814a]
    genotype  [md5: 963a320c8a43d356f14aefde4c26fec0]
    phase  [md5: 29291a178d8ef139e557684563eab2f3]
    annotation/id  [md5: 58b1a28ba2f31cbe877b136657c4ae2d]
    annotation/qual  [md5: 997e0f77c22e2c0bbb2254df87f9cdd4]
    annotation/filter  [md5: 266143e1c3c54b47d6f922d8050fd190]
    annotation/info/PR  [md5: 1431c8252a622660e7678621f16ee720]
Done.
Optimize the access efficiency ...
Clean up the fragments of GDS file:
    open the file '/rds/general/user/ah3918/projects/puklandmarkproject/live/Users/Alex/pipelines/TEST_DATA/test_geno.gds' (84.5K)
    # of fragments: 91
    save to '/

In [28]:
annot<-SeqArray::seqGetData(genofile,"annotation/id")
sample.id<-SeqArray::seqGetData(genofile,"sample.id")
geno_mat<-t(SeqArray::seqGetData(genofile,"$dosage"))
position<-SeqArray::seqGetData(genofile,"position")
chrom<-SeqArray::seqGetData(genofile,"chromosome")

In [29]:
head(geno_mat)

2,2,2,2,2,2,2,2,1,2,⋯,2,1,2,2,2,2,2,2,2,2
0,0,1,1,1,1,1,0,1,1,⋯,0,1,0,0,0,0,0,1,0,0
2,2,1,2,2,2,2,2,1,2,⋯,2,1,2,2,2,2,2,2,2,2
2,2,1,2,2,2,2,2,1,2,⋯,2,1,2,2,2,2,2,2,2,2
2,2,1,2,2,2,1,2,1,2,⋯,2,1,2,2,2,2,2,2,2,2
0,0,1,1,1,1,1,0,1,1,⋯,0,1,0,0,0,0,0,1,0,0


In [20]:
  allele<-SeqArray::seqGetData(genofile,"allele")
  allele<-strsplit(allele,",")
  allele<-as.data.frame(do.call(rbind,allele))

In [ ]:
head(annot)
head

[1] "chr1:758351:A:G"                 "chr1:794299:C:G;chr1:794299:G:C"
[3] "chr1:796338:T:C"                 "chr1:798969:T:C"                
[5] "chr1:800909:T:A"                 "chr1:817186:G:A;chr1:817186:A:G"

In [7]:
 chrompos_mat<-data.frame(annot,chrom,position)
  chrompos_mat<-chrompos_mat[!duplicated(chrompos_mat$annot),]

  #clean / reformat
  colnames(geno_mat)<-sample.id
  geno_mat<-as.data.frame(geno_mat)
  geno_mat$snp<-annot
  geno_mat<-geno_mat[!duplicated(geno_mat$snp),]
  rownames(geno_mat)<-geno_mat$snp

In [27]:
chrompos_mat <- data.frame(annot, chrom, position) %>%
    distinct(annot, .keep_all = TRUE)

geno_mat <- t(geno_mat) %>%
    as.data.frame() %>%
    setNames(sample.id) %>%
    mutate(snp = annot) %>%
    distinct(snp, .keep_all = TRUE) 



ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[33m![39m Can't transform a data frame with `NA` or `""` names.


In [24]:
length(grep(";",chrompos_mat$annot))

[1] 1335004

In [25]:
head(chrompos_mat)
head(annot)

,annot,chrom,position
,<chr>,<chr>,<int>
1,chr1:758351:A:G,1,758351
2,chr1:794299:C:G;chr1:794299:G:C,1,794299
3,chr1:796338:T:C,1,796338
4,chr1:798969:T:C,1,798969
5,chr1:800909:T:A,1,800909
6,chr1:817186:G:A;chr1:817186:A:G,1,817186


[1] "chr1:758351:A:G"                 "chr1:794299:C:G;chr1:794299:G:C"
[3] "chr1:796338:T:C"                 "chr1:798969:T:C"                
[5] "chr1:800909:T:A"                 "chr1:817186:G:A;chr1:817186:A:G"

In [4]:
get_genotype_matrix=function(vcfs,
  gds_file="merge_test_seqArray.gds",
  outdir=".",
  parallel=FALSE,
  preprocess=FALSE,
  autosomalonly=TRUE,
  minmaf=0.05){



  if(preprocess==TRUE){
    SeqArray::seqVCF2GDS(vcfs, gds_file,parallel=parallel)
  }

  genofile<<-SeqArray::seqOpen(gds_file)

  if(autosomalonly==TRUE){
    SeqArray::seqSetFilterChrom(genofile,1:22)
  }

  SeqArray::seqSetFilterCond(genofile,maf=minmaf)

  #get snp info
  annot<-SeqArray::seqGetData(genofile,"annotation/id")

  #get sample names
  sample.id<-SeqArray::seqGetData(genofile,"sample.id")

  #get genotype dosage matrix (0,1,2)
  geno_mat<-t(SeqArray::seqGetData(genofile,"$dosage"))

  #create SNP locations file
  position<-SeqArray::seqGetData(genofile,"position")
  chrom<-SeqArray::seqGetData(genofile,"chromosome")
  chrompos_mat<-data.frame(annot,chrom,position)
  chrompos_mat<-chrompos_mat[!duplicated(chrompos_mat$annot),]

  #clean / reformat
  colnames(geno_mat)<-sample.id
  geno_mat<-as.data.frame(geno_mat)
  geno_mat$snp<-annot
  geno_mat<-geno_mat[!duplicated(geno_mat$snp),]
  rownames(geno_mat)<-geno_mat$snp

  #
  chrompos_mat$chrom<-paste0("chr",chrompos_mat$chrom)
  snpnumber<-length(rownames(geno_mat))

  message("Checking snps for build and converting to rsids..")

  
  chrompos_mat=check_snps(chrompos_mat)
  geno_mat=geno_mat[match(chrompos_mat$old_snp,geno_mat$snp),]
  geno_mat$snp=chrompos_mat$annot
  rownames(geno_mat)=geno_mat$snp
  message(paste0("A total of ",snpnumber," snps were kept."))

  data.table::fwrite(geno_mat,"genotype_012mat.csv")
  

  #add alt. Allele Freq
  allele<-SeqArray::seqGetData(genofile,"allele")
  allele<-strsplit(allele,",")
  allele<-as.data.frame(do.call(rbind,allele))


    
  #check if MAF info is available. If not, calculate it
  maf_avail<-tryCatch({
        out<-SeqArray::seqGetData(genofile,"annotation/info/MAF")
    },
     error=function(e){
        out<-"error"
         return(out)
    }
  )

  if(length(maf_avail)>1){
      maf<-SeqArray::seqGetData(genofile,"annotation/info/MAF")
      }else if(maf_avail=="error"){
      maf<- SeqArray::seqAlleleFreq(genofile, minor=TRUE)
  }
  
  af_df<-data.frame(ref=allele$V1,
    alt=allele$V2,
    maf=maf,
    snp=annot)
    af_df$ref<-as.character(af_df$ref)
    af_df$alt<-as.character(af_df$alt)
    af_df$snp<-as.character(af_df$snp)


    af_df<-af_df[match(chrompos_mat$old_snp,af_df$snp),]
    af_df$snp<-chrompos_mat$annot



      #remove extra columns
  chrompos_mat=chrompos_mat[,c("annot","chrom","position")]
  data.table::fwrite(chrompos_mat,"snp_chromlocations.csv")



  data.table::fwrite(af_df,"MAF_mat.csv")

  SeqArray::seqClose(genofile)
}